# Integrating 2-forms

In this notebook we show how to integrate $2$- forms in $\mathbb{R}^n$ over a $2$-simplex. A $2$-form $\omega \in\Omega^2(\mathbb{R}^n)$ can be written in coordinates as $\omega = \sum_{0\leq i < j \leq n} f_{i,j} dx_i\wedge dx_j$

In [1]:
import numpy as np
import matplotlib as mpl
import torch
import torch.nn as nn
from matplotlib import pyplot as plt
from matplotlib import colors as mcolors
import math
import gudhi as gd 
import scipy.special

#### 1. Subdivide the standard 2-simplex 

The vertices of the standard 2-simplex are ordered first by increasing $x$-value then by increasing $y$-value

In [2]:
# rewrite the function above so that the output is a torch tensor
def subdivide_torch(n):
    # create a list of vertices
    vertices = []
    for i in range(n+1):
        for j in range(n+1-i):
            vertices.append([i/n,j/n])
    vertices = torch.tensor(vertices)
    return vertices

subdivide_torch(2)

tensor([[0.0000, 0.0000],
        [0.0000, 0.5000],
        [0.0000, 1.0000],
        [0.5000, 0.0000],
        [0.5000, 0.5000],
        [1.0000, 0.0000]])

In [3]:
# check if a point is strictly the interior of a triangle with vertices (0,0), (1,0), (0,1)
def is_interior(point):
    if point[0] > 0 and point[1] > 0 and point[0] + point[1] < 1:
        return True
    else:
        return False
    

# check if a point is striclty on the intetior one of the edges of a triangle with vertices (0,0), (1,0), (0,1)
def is_edge(point):
    if point[0] == 0 and point[1] > 0 and point[1] < 1:
        return True
    elif point[0] == 1 and point[1] > 0 and point[1] < 1:
        return True
    elif point[0] > 0 and point[0] < 1 and point[1] == 0:
        return True
    else:
        return False
    

#check if a point is a vertex of a triangle with vertices (0,0), (1,0), (0,1)
def is_vertex(point):
    if point[0] == 0 and point[1] == 0:
        return True
    elif point[0] == 1 and point[1] == 0:
        return True
    elif point[0] == 0 and point[1] == 1:
        return True
    else:
        return False

In [4]:
## Example of using the functions above 

vertices = subdivide_torch(2)
for v in vertices: 
    if is_interior(v):
        print(v, "is in the triangle")
    if is_edge(v):
        print(v, "is on the edge")
    if is_vertex(v):
        print(v, "is a vertex")
    

tensor([0., 0.]) is a vertex
tensor([0.0000, 0.5000]) is on the edge
tensor([0., 1.]) is a vertex
tensor([0.5000, 0.0000]) is on the edge
tensor([1., 0.]) is a vertex


#### 2. Sum a function $g$ over the subdivision of the standard 2-simplex

This will be used later on in integration

In [5]:
## function to integrate a function g over the triangle with vertices (0,0), (1,0), (0,1)

def sum_points(n, g):
    val = 0
    vertices = subdivide_torch(n)
    for pt in vertices:
        if is_interior(pt):
            cof = 6
        elif is_edge(pt):
            cof = 3
        else: 
            cof = 1

        val += cof * g(pt)
    vol = 1/(2*(n**2))
    num_triangles = n**2 
    return  (val* vol )/num_triangles



In [6]:
g = nn.Sequential(
    nn.Linear(2, 1)
)
g(torch.tensor([0.5,0.5]).float())
sum_points(2, g)

tensor([0.0248], grad_fn=<DivBackward0>)

#### 3. Compute the determinant for the sum $g(x)$


the matrix phi embedding the standard 2-simplex is a ($n$ -by- number of vertices in the subdivision of the $2$-simplex)-matrix containing the embedding values in $\mathbb{R}^n$ of each subdivision vertex
the first entry is the embedding of the vertex $(0,0)$ and the order of the vertices follows that given at part 1.

In [7]:
## 2-form f 

n = 4
k = 2
N = int(scipy.special.binom(n,k))
print(N)

kform = nn.Sequential(
    nn.Linear(n, N)
)
kform(torch.tensor([0.5,0.5,0.5,0.4]).float())


6


tensor([ 0.2467, -0.2505, -0.6737,  0.2158,  0.1581,  0.0716],
       grad_fn=<AddBackward0>)

In [8]:
## embedding of the nodes of a triangle in R^4
phi = torch.tensor([[0,0,0,2],[2,1,3,2],[1,2,3,2]])  ## embedding matrix of a triangle in R^2 
print(phi.shape)
print(phi)

torch.Size([3, 4])
tensor([[0, 0, 0, 2],
        [2, 1, 3, 2],
        [1, 2, 3, 2]])


In [9]:
## embed the points of the subdivided triangle in R^4 using the embedding matrix phi
def build_PHI(phi): 
    PHI = torch.zeros((phi.shape[0]-1, phi.shape[1]))
    for i in range(1,phi.shape[0]):
        PHI[i-1] = phi[i] - phi[0]

    #return transpose of PHI
    return PHI.transpose(0,1)



PHI = build_PHI(phi)
print(PHI) 

tensor([[2., 1.],
        [1., 2.],
        [3., 3.],
        [0., 0.]])


In [10]:
# build a tensor of shape (n choose k, n, n) that will be used to compute the determinant

def build_big_deter_tensor(n, k =2):
    N = int(scipy.special.binom(n,k))
    deter_tensor = torch.zeros(N, n, n)
    ind = 0
    for i in range(n):
        for j in range(i+1,n):
            # each tensor is a nxn matrix with (i,j)-coordinate equal to 1 and (j,i)-coordinate equal to -1
            deter_tensor[ind,i,j] = torch.tensor(1)
            deter_tensor[ind,j,i] = torch.tensor(-1)
            ind += 1

     
    return deter_tensor

deter_tensor = build_big_deter_tensor(4,2)
#print(deter_tensor)


In [11]:
# transspose PHI[:,0] to get a column vector
print(PHI[:,0].shape)
print(PHI[:,0].unsqueeze(1).shape)

PHI[:,0].unsqueeze(1).transpose(0,1)

torch.Size([4])
torch.Size([4, 1])


tensor([[2., 1., 3., 0.]])

In [12]:

p = torch.tensor([0.5,0.5]) # point in the subdivided triangle

# build function that computes g
# p a point in the sbdivided triangle

def g(p, kform, phi, deter_tensor):
    PHI = build_PHI(phi)
    p_emb = PHI @ p
    k_form_val = kform(p_emb)

    # make a vector containing all values of det 
    det = torch.zeros(deter_tensor.shape[0])

    for i in range(deter_tensor.shape[0]):
        #det = PHI[0].transpose(0,1) @ deter_tensor[i] @ PHI[1]
        print(PHI[:,0].unsqueeze(1).transpose(0,1).shape)
        print(deter_tensor[i].shape)
        print(PHI[:,1].unsqueeze(1).shape)
        
        det[i] = torch.matmul(torch.matmul(PHI[:,0].unsqueeze(1).transpose(0,1), deter_tensor[i]), PHI[:,1].unsqueeze(1))

    g_val = k_form_val * det
    g_p = torch.sum(g_val)
    return g_p

print(g(p, kform, phi, deter_tensor))



torch.Size([1, 4])
torch.Size([4, 4])
torch.Size([4, 1])
torch.Size([1, 4])
torch.Size([4, 4])
torch.Size([4, 1])
torch.Size([1, 4])
torch.Size([4, 4])
torch.Size([4, 1])
torch.Size([1, 4])
torch.Size([4, 4])
torch.Size([4, 1])
torch.Size([1, 4])
torch.Size([4, 4])
torch.Size([4, 1])
torch.Size([1, 4])
torch.Size([4, 4])
torch.Size([4, 1])
tensor(-6.1173, grad_fn=<SumBackward0>)


In [13]:
def integral_simplex(kform, phi, deter_tensor, n, pt):
    vertices = subdivide_torch(n)
    val = 0
    # compute g at each vertex
    for pt in vertices:
        if is_interior(pt):
            cof = 6
        elif is_edge(pt):
            cof = 3
        else: 
            cof = 1

        val += cof * g(pt, kform, phi, deter_tensor)

    vol = 1/(2*(n**2))

    num_triangles = n**2 

    return  (val* vol )/num_triangles

  
integral_simplex(kform, phi, deter_tensor, 2, p)


torch.Size([1, 4])
torch.Size([4, 4])
torch.Size([4, 1])
torch.Size([1, 4])
torch.Size([4, 4])
torch.Size([4, 1])
torch.Size([1, 4])
torch.Size([4, 4])
torch.Size([4, 1])
torch.Size([1, 4])
torch.Size([4, 4])
torch.Size([4, 1])
torch.Size([1, 4])
torch.Size([4, 4])
torch.Size([4, 1])
torch.Size([1, 4])
torch.Size([4, 4])
torch.Size([4, 1])
torch.Size([1, 4])
torch.Size([4, 4])
torch.Size([4, 1])
torch.Size([1, 4])
torch.Size([4, 4])
torch.Size([4, 1])
torch.Size([1, 4])
torch.Size([4, 4])
torch.Size([4, 1])
torch.Size([1, 4])
torch.Size([4, 4])
torch.Size([4, 1])
torch.Size([1, 4])
torch.Size([4, 4])
torch.Size([4, 1])
torch.Size([1, 4])
torch.Size([4, 4])
torch.Size([4, 1])
torch.Size([1, 4])
torch.Size([4, 4])
torch.Size([4, 1])
torch.Size([1, 4])
torch.Size([4, 4])
torch.Size([4, 1])
torch.Size([1, 4])
torch.Size([4, 4])
torch.Size([4, 1])
torch.Size([1, 4])
torch.Size([4, 4])
torch.Size([4, 1])
torch.Size([1, 4])
torch.Size([4, 4])
torch.Size([4, 1])
torch.Size([1, 4])
torch.Size([

tensor(-1.0521, grad_fn=<DivBackward0>)

# integrate over and entire simplicial complex embedded in $\mathbb{R}^n$

In [14]:
## integrate a k-form over a simplicial complex
SC = gd.SimplexTree()
SC.insert([0,1,2])
SC.insert([1,2,3])

# make a list cotaining all 2-simplices
simplices = []
for simplex in SC.get_skeleton(2):
    print(simplex)
    if len(simplex[0]) == 3:
        print(simplex)
        simplices.append(simplex[0])

print(simplices)

([0, 1, 2], 0.0)
([0, 1, 2], 0.0)
([0, 1], 0.0)
([0, 2], 0.0)
([0], 0.0)
([1, 2, 3], 0.0)
([1, 2, 3], 0.0)
([1, 2], 0.0)
([1, 3], 0.0)
([1], 0.0)
([2, 3], 0.0)
([2], 0.0)
([3], 0.0)
[[0, 1, 2], [1, 2, 3]]


In [16]:
dim = 3 # dim of embedding space
k = 2 # 2-form , 2-simplex
N = int(scipy.special.binom(dim,k))

# build matrix phi witht embedding of all the vertices of the simplicial complex in R^3
phi = torch.tensor([[0,0,0],[1,0,3],[2,1,3],[1,2,1]])
print(phi.shape)
print(phi)

deter_tensor = build_big_deter_tensor(3,2) 
print(deter_tensor)


kform = nn.Sequential(
    nn.Linear(dim, N)
)
kform(torch.tensor([0.5,0.5,0.5]).float())



torch.Size([4, 3])
tensor([[0, 0, 0],
        [1, 0, 3],
        [2, 1, 3],
        [1, 2, 1]])
tensor([[[ 0.,  1.,  0.],
         [-1.,  0.,  0.],
         [ 0.,  0.,  0.]],

        [[ 0.,  0.,  1.],
         [ 0.,  0.,  0.],
         [-1.,  0.,  0.]],

        [[ 0.,  0.,  0.],
         [ 0.,  0.,  1.],
         [ 0., -1.,  0.]]])


tensor([ 0.5123, -0.2366,  0.1990], grad_fn=<AddBackward0>)

In [17]:
# integrate a 2-form over the simplicial complex
def integral_SC(kform, phi, simplices, dim, num_sub):
    deter_tensor = build_big_deter_tensor(dim,2)
    print(deter_tensor) 
    val = 0
    vertices = subdivide_torch(num_sub)
    for simplex in simplices:
        phi_simplex = torch.zeros((len(simplex), n))
        phi_simplex = phi[torch.tensor(simplex)]
        print(phi_simplex)
        for v in vertices:
            val += integral_simplex(kform, phi_simplex, deter_tensor, num_sub, v)
    return val


In [18]:
integral_SC(kform, phi, simplices, dim, 2 )

tensor([[[ 0.,  1.,  0.],
         [-1.,  0.,  0.],
         [ 0.,  0.,  0.]],

        [[ 0.,  0.,  1.],
         [ 0.,  0.,  0.],
         [-1.,  0.,  0.]],

        [[ 0.,  0.,  0.],
         [ 0.,  0.,  1.],
         [ 0., -1.,  0.]]])
tensor([[0, 0, 0],
        [1, 0, 3],
        [2, 1, 3]])
torch.Size([1, 3])
torch.Size([3, 3])
torch.Size([3, 1])
torch.Size([1, 3])
torch.Size([3, 3])
torch.Size([3, 1])
torch.Size([1, 3])
torch.Size([3, 3])
torch.Size([3, 1])
torch.Size([1, 3])
torch.Size([3, 3])
torch.Size([3, 1])
torch.Size([1, 3])
torch.Size([3, 3])
torch.Size([3, 1])
torch.Size([1, 3])
torch.Size([3, 3])
torch.Size([3, 1])
torch.Size([1, 3])
torch.Size([3, 3])
torch.Size([3, 1])
torch.Size([1, 3])
torch.Size([3, 3])
torch.Size([3, 1])
torch.Size([1, 3])
torch.Size([3, 3])
torch.Size([3, 1])
torch.Size([1, 3])
torch.Size([3, 3])
torch.Size([3, 1])
torch.Size([1, 3])
torch.Size([3, 3])
torch.Size([3, 1])
torch.Size([1, 3])
torch.Size([3, 3])
torch.Size([3, 1])
torch.Size([1, 3])

tensor(2.2755, grad_fn=<AddBackward0>)